In [ ]:
""" The Goal here is to Genarate a signal tabel. from the signal clmn
    which includes
    1. Genarate Sl
    2. Genarate Tp < hardCoded - Dinamic is comming in next version
 """

In [ ]:
""" IMPORTING all"""

import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.patches as mpatches

from resampler import EnterpriseDataResampler
from tafm import create_analyzer, IndicatorConfig
from ChartterX5 import Chartter

In [ ]:
import pandas as pd

# Read the saved parquet file
ohlcv = pd.read_parquet("ohlcv-1M-mt5.parquet")

In [ ]:
ohlcv = ohlcv.rename(columns={
    "tickvol": "vol"  # fix spelling too
})
ohlcv.head(3)


In [ ]:
resampler = EnterpriseDataResampler()

dta = resampler.resample_data(
        resampler.filter_business_data(
            resampler.date_filter.filter_by_date_range(
                ohlcv,
                start_date="2024-10-01",
                end_date="2024-10-02"
            ),
            business_hours_only=False,
            weekdays_only=True
        ),
        period="1T"
    ).set_index("datetime")

In [ ]:
"""Applying tecNical Ananlisis.."""
dta = dta.rename(columns={
    "open": "Open",
    "high": "High",
    "low": "Low",
    "close": "Close",
    "vol": "Volume"  # fix spelling too
})

analyzer = create_analyzer(dta)

analyzer.add_indicator(IndicatorConfig(name='RSI', period=14, source_column='Close'))
analyzer.add_indicator(IndicatorConfig(name='EMA', period=15, source_column='RSI_14'))


In [ ]:
#>=Signal Genarator

analyzer.crossed_up('RSI_14', 'EMA_15')

analyzer.df.tail(10)

In [ ]:
ohlcv = analyzer.df.rename(columns={
    "Open": "open",
    "High": "high",
    "Low": "low",
    "Close": "close",
    "Volume": "volume"  # fix spelling too
})


In [ ]:
# Initialize chartter with wider and less tall proportions, no volume
chartter = Chartter(config={
    'chart_type': 'candle',
    'style': 'charles',
    'figratio': (20, 8),  # Adjusted for wider and less tall chart
    'volume': False  # Disable volume bars
})

# --- TradingView Market Colors ---
tv_mc = mpf.make_marketcolors(
    up='#26a69a',         # TradingView teal green for up candles
    down='#ef5350',       # TradingView red for down candles
    edge='inherit',       # Clean edges matching candle color
    wick='inherit',       # Wicks match candle colors
)

# --- TradingView Dark Theme ---
tradingview_dark = mpf.make_mpf_style(
    base_mpf_style='nightclouds',  # Start with dark base
    marketcolors=tv_mc,

    # TradingView dark theme colors
    figcolor='#131722',           # Dark navy background
    facecolor='#1e222d',          # Dark gray chart area
    gridcolor='#363a45',          # Dark gray grid
    gridstyle='-',                # Solid grid lines

    y_on_right=True,              # Price axis on right

    rc={
        # TradingView typography
        # 'font.family': ['-apple-system', 'BlinkMacSystemFont', 'Trebuchet MS', 'Roboto', 'Ubuntu', 'sans-serif'],
        'axes.labelsize': 10,
        'axes.titlesize': 14,
        'xtick.labelsize': 9,
        'ytick.labelsize': 7,         # Reduced size for price tick labels
        'legend.fontsize': 9,

        # Clean spacing with reduced left padding and full-width chart
        'axes.labelpad': 10,
        'xtick.major.pad': 6,
        'ytick.major.pad': 6,
        'axes.xmargin': 0,            # Remove left/right margins
        'axes.ymargin': 0,            # Remove top/bottom margins
        'figure.subplot.left': 0.05,  # Minimize left subplot padding
        'figure.subplot.right': 0.95, # Maximize right subplot to fit price scale

        # TradingView-style lines
        'lines.linewidth': 1.5,
        'lines.antialiased': True,

        # Clean borders and colors
        'axes.edgecolor': '#434651',  # Darker edge for dark theme
        'axes.linewidth': 1,
        'xtick.color': '#787b86',     # Light gray for x-axis ticks
        'ytick.color': '#787b86',     # Light gray for y-axis ticks
        'axes.labelcolor': 'none',    # Hide axis label color (labels are empty)

        # Grid styling
        'axes.grid': True,
        'axes.axisbelow': True,
        'grid.alpha': 0.6,            # Slightly more transparent for dark theme
        'grid.linewidth': 0.8,

        # Clean spines (show right spine for price scale)
        'axes.spines.left': True,
        'axes.spines.bottom': True,
        'axes.spines.top': False,
        'axes.spines.right': True,    # Show right spine for price scale
        'ytick.right': True,          # Show right y-axis ticks
    }
)

#tradingview_config_4H
tradingview_config_4H = {
    'title': 'XAUUSD, 4H',                # TradingView style title
    'style': tradingview_dark,             # Use dark theme
    'type': 'candle',
    'volume': False,                      # Disable volume bars
    'show_nontrading': False,             # Ensure no extra y-axis spacing
    'datetime_format': '%Y-%m-%d',        # Format x-axis to show date only
    'xlabel': '',                         # Remove x-axis label
    'ylabel': '',                         # Remove y-axis label
    'xrotation': 0                        # Horizontal date labels
}


In [ ]:
dpart = ohlcv[:300]

In [ ]:
fig, axes = chartter.plot(
    dpart,
    addplot = [
        mpf.make_addplot(dpart['RSI_14'], panel=1, color='purple', ylabel='RSI'),
        mpf.make_addplot(dpart['EMA_15'], panel=1, color='blue', ylabel='ema15'),
    ],
    config=tradingview_config_4H,
    vlines=dict(
        vlines=dpart.index[dpart['RSI_14_crossed_up_EMA_15'] == 1].to_list(),
        colors='white',
        linewidths=0.3,
        alpha=0.1
    ),
    returnfig=True
)

In [ ]:
dta = dpart.copy()


dta['SL'] = dta['low'] - (dta ['high'] - dta ['low'])


dta['TP'] = dta['close'] + (dta ['high'] - dta ['low']) * 2


# dta.head(3)

In [ ]:
# Get row numbers (integer positions) where signal is 1
row_numbers = np.where(dta['RSI_14_crossed_up_EMA_15'] == 1)[0]

# Extract SL and TP values for those rows
signal = dta[dta['RSI_14_crossed_up_EMA_15'] == 1]
Close_Arr = signal['close'].to_numpy()
sl_array = signal['SL'].to_numpy()
tp_array = signal['TP'].to_numpy()

# Optional: combine into structured array
signal_array = np.array(list(zip(row_numbers,Close_Arr, sl_array, tp_array)),
                        dtype=[('row', int), ('close', float), ('SL', float), ('TP', float)])

In [ ]:
def draw_risk_reward_boxes(ax, trades_array):
    for trade in trades_array:
        row = trade['row']
        close_price = trade['close']
        sl = trade['SL']
        tp = trade['TP']

        # --- Risk box: close → SL ---
        risk_bottom = min(close_price, sl)
        risk_height = abs(close_price - sl)
        risk_rect = mpatches.Rectangle(
            (row - 0.5, risk_bottom),
            1,                 # width = 1 candle
            risk_height,
            color='red',
            alpha=0.3
        )
        ax.add_patch(risk_rect)

        # --- Reward box: close → TP ---
        reward_bottom = min(close_price, tp)
        reward_height = abs(close_price - tp)
        reward_rect = mpatches.Rectangle(
            (row - 0.5, reward_bottom),
            1,
            reward_height,
            color='green',
            alpha=0.3
        )
        ax.add_patch(reward_rect)


In [ ]:
fig, axes = chartter.plot(
    dpart,
    addplot = [
        mpf.make_addplot(dpart['RSI_14'], panel=1, color='purple', ylabel='RSI'),
        mpf.make_addplot(dpart['EMA_15'], panel=1, color='blue', ylabel='ema15'),
    ],
    config={
    'title': ' ',
    'style': tradingview_dark,
    'volume': False,                      # Disable volume bars
    'show_nontrading': False,             # Ensure no extra y-axis spacing
    'datetime_format': '%Y-%m-%d',        # Format x-axis to show date only
    'xlabel': '',                         # Remove x-axis label
    'ylabel': '',                         # Remove y-axis label
    'xrotation': 0                        # Horizontal date labels
},
    returnfig=True
)

draw_risk_reward_boxes(axes[0], signal_array)

## Backtest Reports - 

In [ ]:
from bktst import EnterpriseTradingFramework

framework = EnterpriseTradingFramework()

# Example usage (commented)
results = framework.run_backtest(dta, commission = 0)


In [ ]:
trades = results['trades_table']


row_numbers = dta.index.get_indexer(trades['datetime_entry'])

# Entry/Exit prices
entry_array = trades['entry_price'].to_numpy()
exit_array = trades['exit_price'].to_numpy()

# SL/TP prices
sl_array = trades['sl_level'].to_numpy()  # assuming trades also has SL
tp_array = trades['tp_level'].to_numpy()  # assuming trades also has TP

# Structured arrays
entry_exit_arr = np.zeros(len(trades), dtype=[('row', int), ('entry', float), ('exit', float)])
entry_exit_arr['row'] = row_numbers
entry_exit_arr['entry'] = entry_array
entry_exit_arr['exit'] = exit_array

sltp_arr = np.zeros(len(trades), dtype=[('row', int), ('SL', float), ('TP', float)])
sltp_arr['row'] = row_numbers
sltp_arr['SL'] = sl_array
sltp_arr['TP'] = tp_array

In [ ]:
import matplotlib.patches as mpatches

def draw_boxes_border_only(ax, arr, y1_col, y2_col, edge_color='red', lw=1.5, alpha=1.0):
    for row_data in arr:
        row = row_data['row']
        y1 = row_data[y1_col]
        y2 = row_data[y2_col]
        rect = mpatches.Rectangle(
            (row - 0.5, y1),  # left bottom corner
            1,                # width = 1 candle
            y2 - y1,          # height
            facecolor='none', # no fill
            edgecolor=edge_color,
            linewidth=lw,
            alpha=alpha,
            zorder=10         # Ensure it's drawn on top
        )
        ax.add_patch(rect)

fig, axes = chartter.plot(
    dpart,
    addplot = [
        mpf.make_addplot(dpart['RSI_14'], panel=1, color='purple', ylabel='RSI'),
        mpf.make_addplot(dpart['EMA_15'], panel=1, color='blue', ylabel='ema15'),
    ],
    config=tradingview_config_4H,
    returnfig=True
)

# Draw SL/TP boxes with border only
draw_boxes_border_only(axes[0], sltp_arr, 'SL', 'TP', edge_color='white', lw=0.5)

# Draw Entry/Exit boxes with border only
draw_boxes_border_only(axes[0], entry_exit_arr, 'entry', 'exit', edge_color='red', lw=0.5)


In [ ]:
results['account_summary']

In [ ]:
results['trades_table'].head(3)
#dict_keys(['orders_table', 'trades_table', 'account_summary', 'daily_summary', 'analyzers', 'strategy'])

## Save dta for future ananalisis

In [ ]:
# Save DataFrame as Parquet
#dta.to_parquet("sampleSignal.parquet", index=True)  # index=True to keep datetime index
